# Data preparation fro DIGITS

Prepare data as described [here](https://github.com/NVIDIA/DIGITS/blob/master/digits/extensions/data/objectDetection/README.md) to create dataset for object detection task

In [1]:
# Python
import sys
import json
import os
from os.path import join
import shutil
# Numpy
import numpy as np

In [2]:
from PIL import Image

In [3]:
def get_annotations(filename):
    """
    :return: ndarray of dicts 
        {
            "annotations": [
                {
                    "class": "os",
                    "height": 10.0,
                    "type": "rect",
                    "width": 20.0,
                    "x": 52.0,
                    "y": 48.0
                },
                {
                    "class": "cervix",
                    "height": 275.0,
                    "type": "rect",
                    "width": 300.0,
                    "x": 10.0,
                    "y": 5.0
                }],
            "class": "image",
            "filename": "train/Type_1/590.jpg"
        }
    """
    labels = []
    with open(filename, 'r') as reader:
        str_data = ''.join(reader.readlines())
        raw_data = json.loads(str_data)
        for item in raw_data:
            if len(item['annotations']) > 0:
                labels.append(item)
    return np.array(labels)
   
    
def write_images_labels(annotations, data_path, output_path, create_sym_links=True):
    """ 
        LABEL STRUCTURE from DIGITS\digits\extensions\data\objectDetection\utils.py
    
        This class is the data ground-truth

        #Values    Name      Description
        ----------------------------------------------------------------------------
        1    type         Class ID
        1    truncated    Float from 0 (non-truncated) to 1 (truncated), where
                          truncated refers to the object leaving image boundaries.
                          -1 corresponds to a don't care region.
        1    occluded     Integer (-1,0,1,2) indicating occlusion state:
                          -1 = unkown, 0 = fully visible,
                          1 = partly occluded, 2 = largely occluded
        1    alpha        Observation angle of object, ranging [-pi..pi]
        4    bbox         2D bounding box of object in the image (0-based index):
                          contains left, top, right, bottom pixel coordinates
        3    dimensions   3D object dimensions: height, width, length (in meters)
        3    location     3D object location x,y,z in camera coordinates (in meters)
        1    rotation_y   Rotation ry around Y-axis in camera coordinates [-pi..pi]
        1    score        Only for results: Float, indicating confidence in
                          detection, needed for p/r curves, higher is better.

        Here, 'DontCare' labels denote regions in which objects have not been labeled,
        for example because they have been too far away from the laser scanner.
    """
    output_images_folder = os.path.join(output_path, "images")
    os.makedirs(output_images_folder)
    
    output_labels_folder = os.path.join(output_path, "labels")
    os.makedirs(output_labels_folder)
    
    def _clamp(x, dim):
        return min(max(x, 0), dim-1)
        
    for annotation in annotations:
        img_filename = annotation['filename']
        basename, ext = os.path.splitext(os.path.basename(img_filename))
        basename = os.path.split(os.path.dirname(img_filename))[1] + '_' + basename
        src_image_filename = os.path.join(data_path, img_filename)
        dst_image_filename = os.path.join(output_images_folder, "%s%s" % (basename,ext))
        dst_label_filename = os.path.join(output_labels_folder, "%s.txt" % basename)
        
        os.symlink(src_image_filename, dst_image_filename)
        pil_image = Image.open(img_filename)
        image_size = pil_image.size       
        
        with open(dst_label_filename, 'w') as writer:
            for obj in annotation['annotations']:
                # format : class_name bbox_left bbox_top bbox_right bbox_bottom
                l, t, w, h = int(obj['x']), int(obj['y']), int(obj['width']), int(obj['height'])
                r = l+w; b = t+h
                l = _clamp(l, image_size[0])
                t = _clamp(t, image_size[1])
                r = _clamp(r, image_size[0])
                b = _clamp(b, image_size[1])
                line = "{type} {truncated} {occluded} {alpha} {l} {t} {r} {b} {h} {w} {le} {x} {y} {z} {ry}\n".format(
                    type=obj['class'],
                    truncated=0.0,
                    occluded=-1,
                    alpha=0.0,
                    l=l, t=t, r=r, b=b,
                    h=0.0, w=0.0, le=0.0,
                    x=0.0, y=0.0, z=0.0,
                    ry = 0.0
                )   
                writer.write(line)

In [4]:
RAW_DATA_PATH = os.path.join('..')
SLOTH_LABELS_PATH = os.path.join('..', 'resources', 'cervix_os.json')
TRAIN_TEST_SPLIT=0.7

In [5]:
annotations = get_annotations(SLOTH_LABELS_PATH)

# Create data split
num_labels = len(annotations)
indices = np.random.permutation(num_labels)
split_index = int(num_labels * TRAIN_TEST_SPLIT)
train_annotations = annotations[indices[:split_index]]
test_annotations = annotations[indices[split_index:]]

print "Total : %s, Train : %s, Val : %s" % (num_labels, len(train_annotations), len(test_annotations))

Total : 208, Train : 145, Val : 63


In [6]:
# train_filenames = [a['filename'] for a in train_annotations]
# test_filenames = [a['filename'] for a in test_annotations]
# set(train_filenames) & set(test_filenames)

In [7]:
# Following DIGITS conventions : Label files are expected to have the .txt extension. 
# For example if an image file is named foo.png the corresponding label file should be foo.txt. 
# And specific for object detection
# https://github.com/NVIDIA/DIGITS/tree/master/digits/extensions/data/objectDetection

GENERATED_DATA=join("..", "input", "generated")

if os.path.isdir(join(GENERATED_DATA, "train")):
    shutil.rmtree(join(GENERATED_DATA, "train"))
if os.path.isdir(join(GENERATED_DATA, "val")):
    shutil.rmtree(join(GENERATED_DATA, "val"))

write_images_labels(train_annotations, RAW_DATA_PATH, join(GENERATED_DATA, "train"))
write_images_labels(test_annotations, RAW_DATA_PATH, join(GENERATED_DATA, "val"))


In [9]:
!ls {GENERATED_DATA}/train/labels | wc -l
!ls {GENERATED_DATA}/val/labels | wc -l

145
63


In [10]:
!ls {GENERATED_DATA}/train/labels/

Type_1_0.txt	 Type_1_376.txt  Type_1_708.txt  Type_1_972.txt
Type_1_1013.txt  Type_1_379.txt  Type_1_709.txt  Type_1_977.txt
Type_1_1014.txt  Type_1_383.txt  Type_1_710.txt  Type_1_998.txt
Type_1_102.txt	 Type_1_387.txt  Type_1_713.txt  Type_2_1197.txt
Type_1_104.txt	 Type_1_396.txt  Type_1_732.txt  Type_2_1198.txt
Type_1_109.txt	 Type_1_401.txt  Type_1_739.txt  Type_2_1203.txt
Type_1_10.txt	 Type_1_41.txt	 Type_1_751.txt  Type_2_1210.txt
Type_1_129.txt	 Type_1_421.txt  Type_1_759.txt  Type_2_1211.txt
Type_1_12.txt	 Type_1_425.txt  Type_1_763.txt  Type_2_1218.txt
Type_1_138.txt	 Type_1_441.txt  Type_1_764.txt  Type_2_1371.txt
Type_1_144.txt	 Type_1_446.txt  Type_1_779.txt  Type_2_1373.txt
Type_1_148.txt	 Type_1_454.txt  Type_1_783.txt  Type_2_1376.txt
Type_1_14.txt	 Type_1_469.txt  Type_1_787.txt  Type_2_1378.txt
Type_1_160.txt	 Type_1_481.txt  Type_1_7.txt	 Type_2_1379.txt
Type_1_171.txt	 Type_1_48.txt	 Type_1_805.txt  Type_2_1415.txt
Type_1_180.txt	 Type_1_536.txt  Type_1_817.txt  Ty

In [15]:
!ls {GENERATED_DATA}/val/labels

Type_1_1019.txt  Type_1_338.txt  Type_1_643.txt  Type_1_901.txt
Type_1_1023.txt  Type_1_384.txt  Type_1_645.txt  Type_2_1195.txt
Type_1_139.txt	 Type_1_470.txt  Type_1_663.txt  Type_2_1196.txt
Type_1_13.txt	 Type_1_471.txt  Type_1_685.txt  Type_2_1201.txt
Type_1_142.txt	 Type_1_47.txt	 Type_1_725.txt  Type_2_1205.txt
Type_1_176.txt	 Type_1_484.txt  Type_1_727.txt  Type_2_1209.txt
Type_1_201.txt	 Type_1_497.txt  Type_1_745.txt  Type_2_1212.txt
Type_1_208.txt	 Type_1_513.txt  Type_1_765.txt  Type_2_1368.txt
Type_1_215.txt	 Type_1_516.txt  Type_1_769.txt  Type_2_1369.txt
Type_1_229.txt	 Type_1_518.txt  Type_1_791.txt  Type_2_1377.txt
Type_1_245.txt	 Type_1_531.txt  Type_1_802.txt  Type_2_1428.txt
Type_1_252.txt	 Type_1_550.txt  Type_1_809.txt  Type_2_1432.txt
Type_1_254.txt	 Type_1_578.txt  Type_1_810.txt  Type_2_1445.txt
Type_1_311.txt	 Type_1_582.txt  Type_1_81.txt	 Type_2_1449.txt
Type_1_333.txt	 Type_1_593.txt  Type_1_833.txt  Type_2_1453.txt
Type_1_334.txt	 Type_1_596.txt  Type_1_836

In [16]:
!cat {GENERATED_DATA}/val/labels/Type_1_1019.txt

os 0.0 -1 0.0 553 1965 1613 2447 0.0 0.0 0.0 0.0 0.0 0.0 0.0
cervix 0.0 -1 0.0 327 1220 2143 3197 0.0 0.0 0.0 0.0 0.0 0.0 0.0
